In [1]:
import numpy as np
import random
from tqdm import tqdm # loops show smart progress meter
from enum import Enum

In [2]:

class Board:
    """ holds a 3x3 board.
    """
        
    @staticmethod
    def coordFromLinearCoord(pos):
        assert(0 <= pos < 9)
        
        x = pos % 3
        y = int(pos/3)
        
        return x, y
    
    @staticmethod
    def coordToLinearCoord(x, y):
        assert(0 <= x < 3)
        assert(0 <= y < 3)
        
        return x + 3 * y
    
    def __init__(self, b = np.zeros((3,3))):
        """ b is a numpy array.
        """
        
        assert(b.shape == ((3,3)))
        
        self.b = b
    
    def hasEmptyCell(self):
        return np.any(self.b == 0) == False
    
    def isEmptyCell(self, x, y):
        return self.b[x, y] == 0
    
    def check(self, player):
        """
        This function returns player if this player won, 2 if draw, and 0 otherwise.
        """
        
        assert(player in [-1, 1])
                
        b = self.b
        
        # check horizontal and vertical lines for win.

        for i in range(3):
            if (b[i,0] == player and b[i,0] == b[i,1] == b[i,2]) or \
               (b[0,i] == player and b[0,i] == b[1,i] == b[2,i]):
                return player

        # check diagonals for win.
        
        if (b[0,0] == player and b[0,0] == b[1,1] == b[2,2]) or \
           (b[0,2] == player and b[0,2] == b[1,1] == b[2,0]):
            return player
        
        
        if self.hasEmptyCell():   # draw.
            return 2
        
        return 0    # game not over.

    def toKey(self):
        
        s = 0
        i = 0
        
        for x in range(3):
            for y in range(3):
                s += (self.b[x,y] + 2) * 10**i
                i += 1

        return int(s)

    def fromKey(self, i):
        
        self.b = np.zeros((3, 3))
        
        for x in range(3):
            for y in range(3):
                self.b[x,y] = i % 10 - 2
                i = int(i/10)
                
    def move(self, x, y, player):
        self.b[x,y] = player
        
        return self.check(player)
    
    def possibleActions(self):
        
        """ returns list of empty cells using linear coords.
        """
        
        return [Board.coordToLinearCoord(x, y) for x in range(3) for y in range(3) if self.isEmptyCell(x, y)]


def getAllStartStates():
    def getStartStates(keys, b, pos):
        # i is initial cell in linear coords.
        
        assert(0 <= pos < 9)
        
        x, y = Board.coordFromLinearCoord(pos)
        
        for p in [-1, 0, 1]:
            b.b[x,y] = p
            
            if np.sum(b.b == 1) - np.sum(b.b == -1) in [0, 1] and b.check(1) == 0 and b.check(-1) == 0:
                keys.append(b.toKey())
            
            if pos < 8:
                getStartStates(keys, b, pos + 1)
    
    board = Board()
    boardKeys = [board.toKey()]
    
    getStartStates(boardKeys, board, 0)
    
    return boardKeys

In [3]:
def policyRandomMove(board, pure = False):
    
    p = np.where(board.b == 0, 1, 0) # p is 3x3 grid which is 1 where we have empty cells
    
    corners = [[0,0], [0,2], [2,0], [2,2]]
    
    if not pure:
        if p[1,1] == 1:  # (1, 1) is empty
            p[1,1] = 3
        for x, y in corners:
            if p[x,y] == 1:
                p[x,y] = 2
    
    policy = p / np.sum(p)
    
    return chooseMove(policy)

# does this function always return something?

def chooseMove(policy):
    
    u = np.random.uniform()
    c = 0

    for x in range(3):
        for y in range(3):
            c += policy[x, y]
            if u < c:
                return x, y

def policyMCMove(board, q, player):
    """
    We choose the action that has the highest current q-value.
    """
    key = board.toKey()
    actions = board.possibleActions()  # actually just empty cells.

    qValues = [q[key, action] for action in actions]

    bestAction = actions[qValues.index(max(qValues))]
    
    return Board.coordFromLinearCoord(bestAction)

def policyMCMoveEpsGreedy(board, q, player, epsilon):
    """
    With probabiliy epsilon we choose an action randomly.
    """
    if random.random() < epsilon:
        a = random.choice(board.possibleActions())
        return Board.coordFromLinearCoord(a)
    
    return policyMCMove(board, q, player)

def expectedTargetUnderEpsGreedy(board, q, epsilon):

    key = board.toKey()
    actions = board.possibleActions()
    
    qValues = [q[(key, a)] for a in actions]
    bestq = max(qValues)
    
    return bestq * (1 - epsilon) + epsilon * sum(qValues) / len(actions)

def playSingleGame(epsilon, q, returnStateActionPairs = True):

    player = random.choice([-1,1])
    board = Board(np.zeros((3,3)))
    
    listStateAction = []
    
    gameState = 0
    
    while gameState == 0:
        if player == 1:
            x, y = policyRandomMove(board)
        else:
            x, y = policyMCMoveEpsGreedy(board, q, player, epsilon)
            if returnStateActionPairs:
                listStateAction.append((board.toKey(), (x, y)))
        
        gameState = board.move(x, y, player)
        player *= -1
        
    return gameState, listStateAction

def playGame(nGames, q, epsilon = -1):
    
    win1 = 0
    winM1 = 0
    draw = 0
    
    for _ in tqdm(range(nGames)):

        gameState, _ = playSingleGame(epsilon, q, False)

        if gameState == 2:
            draw += 1
        elif gameState == 1:
            win1 += 1
        else:
            winM1 += 1

    print(win1/nGames, winM1/nGames, draw/nGames)

In [4]:
def initialise(keys):

    q = {}
    n = {}

    for key in keys:
        board = Board()
        board.fromKey(key)
              
        for action in board.possibleActions():
            q[(key, action)] = 0.5
            n[(key, action)] = 0
    
    return q, n

Here we look at the $\epsilon$-greedy method.  This means that we choose the current best action most of the time but sometimes, namely with probability $\epsilon$, we explore.  In our case, one player plays randomly and the other follows this strategy.  The action-value function $q(s, a)$ is a function of state $s$ and action $a$. 

In [15]:
#MC epsilon greedy

q, n = initialise(getAllStartStates())

epsilon = 0.05
nGames = 100000

for _ in tqdm(range(nGames)):

    finalGameState, listStateAction = playSingleGame(epsilon, q)

    # r is the reward.
    
    if finalGameState == 2: # draw
        r = 0.5
    elif finalGameState == 1: # loss
        r = 0
    else: # win
        r = 1

    for key, (x, y) in listStateAction:
        action = Board.coordToLinearCoord(x, y)
        
        u = (key, action)
        
        q[u] = q[u] * n[u] + r
        n[u] += 1
        q[u] /= n[u]

playGame(nGames, q, epsilon)

100%|██████████| 100000/100000 [00:25<00:00, 3889.58it/s]

0.05351 0.8658 0.08069


In [18]:
s = getAllStartStates()
q, n = initialise(s)

In [19]:
#MC exploring starts
nGames = 200000
for _ in tqdm(range(nGames)):
    b = Board(np.zeros((3,3)))
    b.fromKey(random.choice(s))
    
    player = -1
    
    randomAction = True
    
    gameOver = 0
    
    listStateAction=[]
    
    while gameOver == 0:
        if player == 1:
            x, y = policyRandomMove(b)
        else:
            key = b.toKey()
            if randomAction:
                p = np.where(b.b==0, 1, 0)
                policy = p / np.sum(p)
                x,y = chooseMove(policy)
                randomAction = False
            else:
                x,y = policyMCMove(b, q, player)
            
            listStateAction.append((key, (x, y)))
        
        gameOver = b.move(x, y, player)
        
        player *= -1
        
    if gameOver == 2:
        r = 0.5
    elif gameOver == 1:
        r = 0
    else:
        r = 1
    
    for key, (x,y) in listStateAction:
        a = Board.coordToLinearCoord(x, y)
        
        u = (key, a)
        
        q[u] = q[u] * n[u] + r
        n[u] += 1
        q[u] /= n[u]

playGame(nGames, q)

100%|██████████| 200000/200000 [00:56<00:00, 3511.39it/s]

0.08122 0.85802 0.06076


In [20]:
q, c = initialise(getAllStartStates())

In [21]:
#off policy
nGames = 500000

def playSingleGame(epsilon, returnStateActionPairs = True):
    
    player = random.choice([-1,1])
    board = Board(np.zeros((3,3)))
    
    listStateAction = []
    
    gameState = 0
    
    while gameState == 0:
        if player == 1:
            x, y = policyRandomMove(board)
        else:
            x, y = policyMCMoveEpsGreedy(board, q, player, epsilon)
            if returnStateActionPairs:
                listStateAction.append((board.toKey(), (x, y)))
        
        gameState = board.move(x, y, player)
        player *= -1
        
    return gameState, listStateAction

for _ in tqdm(range(nGames)):
    b = Board(np.zeros((3,3)))
    player = random.choice([-1,1])
    gameOver = 0
    listStateAction=[]
    
    while gameOver==0:
        if player==1:
            x,y=policyRandomMove(b)
        else:
            x,y=policyRandomMove(b, True)
            listStateAction.append((b.toKey(), (x, y)))
        gameOver = b.move(x,y,player)
        player=-player
        
    if gameOver==2:
        r=0.5
    elif gameOver==1:
        r=0
    else:
        r=1

    W=1
    for k, (x,y) in listStateAction[::]:
        a = Board.coordToLinearCoord(x, y)
        c[(k,a)]=c[(k,a)]+W
        q[(k,a)]=q[(k,a)]+W*(r-q[(k,a)])/c[(k,a)]
        b = Board(np.zeros((3,3)))
        b.fromKey(k)
        pA=b.possibleActions()
        bestA=pA[0]
        for aa in pA[1:]:
            if q[(k,aa)]>q[(k,bestA)]:
                bestA=aa
        if bestA!=a:
            break
        W=W*len(pA)

playGame(nGames, q)

100%|██████████| 500000/500000 [02:02<00:00, 4065.32it/s]

0.05966 0.903212 0.037128


In [22]:
q, _ = initialise(getAllStartStates())

In [23]:
#sarsa
nGames = 100000
alpha = 0.1
epsilon = 0.05

for _ in tqdm(range(nGames)):
    b = Board(np.zeros((3,3)))
    player = random.choice([-1,1])
    
    gameOver = 0
    action = None
    newAction = None
    
    while gameOver == 0:
        if player == 1:
            x, y = policyRandomMove(b)
        else:
            x, y = policyMCMoveEpsGreedy(b,q,player,epsilon)
            if action is None:
                action = Board.coordToLinearCoord(x, y)
                key = b.toKey()
            else:
                newAction = Board.coordToLinearCoord(x, y)
                newKey = b.toKey()
                
        gameOver = b.move(x,y,player)
        if gameOver==-1:
            q[(key,action)]=q[(key,action)]+alpha*(1-q[(key,action)])
        elif gameOver==1:
            q[(key,action)]=q[(key,action)]+alpha*(0-q[(key,action)])
        elif player==-1 and newAction is not None:
            q[(key,action)]=q[(key,action)]+alpha*(q[(newKey,newAction)]-q[(key,action)])
            action=newAction
            key=newKey
        player=-player

playGame(nGames, q)

100%|██████████| 100000/100000 [00:36<00:00, 2746.75it/s]

0.04545 0.76722 0.18733


In [24]:
q, _ = initialise(getAllStartStates())

In [25]:
#q learning
nGames = 100000
alpha=0.1
epsilon=0.1
for _ in tqdm(range(nGames)):
    b = Board(np.zeros((3,3)))
    player = random.choice([-1,1])
    gameOver=0
    action=None
    newAction=None
    while gameOver==0:
        if player==1:
            x,y=policyRandomMove(b)
        else:
            x, y = policyMCMoveEpsGreedy(b,q,player,epsilon)
            if action is None:
                action = Board.coordToLinearCoord(x, y)
                key = b.toKey()
            else:
                newAction = Board.coordToLinearCoord(x, y)
                newKey = b.toKey()
                pA=b.possibleActions()
                maxq=q[(newKey,pA[0])]
                for aa in pA[1:]:
                    if q[(newKey,aa)]>maxq:
                        maxq=q[(newKey,aa)]
        gameOver = b.move(x,y,player)
        if gameOver==-1:
            q[(key,action)]=q[(key,action)]+alpha*(1-q[(key,action)])
        elif gameOver==1:
            q[(key,action)]=q[(key,action)]+alpha*(0-q[(key,action)])
        elif player==-1 and newAction is not None:
            q[(key,action)]=q[(key,action)]+alpha*(maxq-q[(key,action)])
            action=newAction
            key=newKey
        player=-player

playGame(nGames, q)

100%|██████████| 100000/100000 [00:33<00:00, 2961.85it/s]

0.03501 0.75657 0.20842


In [26]:
q, _ = initialise(getAllStartStates())

In [27]:
# expected sarsa where behaviour policy is eps greedy derived from q
# (expected sarsa where behaviour policy is greedy derived from q is the same as q learning)
nGames = 100000
alpha = 0.1
epsilon = 0.1

for _ in tqdm(range(nGames)):
    b = Board(np.zeros((3,3)))
    
    player = random.choice([-1,1])
    
    gameOver = 0
    
    action = None
    newAction = None
    
    while gameOver == 0:
        if player == 1:
            x, y = policyRandomMove(b)
        else:
            x, y = policyMCMoveEpsGreedy(b, q, player, epsilon)
            if action is None:
                action = Board.coordToLinearCoord(x, y)
                key = b.toKey()
            else:
                newAction = Board.coordToLinearCoord(x, y)
                newKey = b.toKey()
                expectedTarget = expectedTargetUnderEpsGreedy(b, q, epsilon)
        gameOver = b.move(x,y,player)
        if gameOver==-1:
            q[(key,action)]=q[(key,action)]+alpha*(1-q[(key,action)])
        elif gameOver==1:
            q[(key,action)]=q[(key,action)]+alpha*(0-q[(key,action)])
        elif player==-1 and newAction is not None:
            q[(key,action)]=q[(key,action)]+alpha*(expectedTarget-q[(key,action)])
            action=newAction
            key=newKey
        player=-player

playGame(nGames, q)

100%|██████████| 100000/100000 [00:37<00:00, 2659.04it/s]

0.02518 0.77884 0.19598


Learning improves a policy from experience gathered by interacting with the environment (direct RL). During this learning, a model of the environment can be created by remembering the rewards and new states that followed states and actions. Such a model can be used for planning (indirect RL). Direct and indirect RL are combined in the dyna architectures. If the environment is deterministic, then the model can also be deterministic and sample updates can be used during the planning step. If the environmenet is stochastic, then the model should also be stochastic and normally expected updates are used.

Planning is used to make the learning more sample efficient.

During planning, we have to choose states for the planning step. this state search can be uniformly random (like in dyna) or follow rules, eg prioritized sweeping.

We implement dyna-q as an example. prioritized sweeping seems exaggerated for these short episode with few actions.
We couldnt observe a great improvement in sample efficiency which might be due to the opponent following a random strategy which makes the environment (almost pure) random.

There is another way of planning, sometimes called planning at decision time where the planning focuses on the current state, eg Monte Carlo Tree Search. In this simple example of TicTacToe, such algorithms can compute the optimal fast enough such that they would be a good solution.

In [28]:
q, _ = initialise(getAllStartStates())

In [29]:
#dyna-q (for stoch env)

def updateModel(model, key, newState):
    if key in model:
        dist=model[key]
        if newState in dist:
            dist[newState]+=1
        else:
            dist[newState]=1
    else:
        dist={newState:1}
        model[key]=dist
    
nGames = 100000
alpha=0.1
epsilon=0.1
model={}
hb = Board(np.zeros((3,3)))
nPlanSteps=20
for _ in tqdm(range(nGames)):
    b = Board(np.zeros((3,3)))
    player = random.choice([-1,1])
    gameOver=0
    action=None
    newAction=None
    while gameOver==0:
        if player==1:
            x,y=policyRandomMove(b)
        else:
            x, y = policyMCMoveEpsGreedy(b,q,player,epsilon)
            if action is None:
                action = x+3*y
                key = b.toKey()
            else:
                newAction = x+3*y
                newKey = b.toKey()
                pA=b.possibleActions()
                maxq=q[(newKey,pA[0])]
                for aa in pA[1:]:
                    if q[(newKey,aa)]>maxq:
                        maxq=q[(newKey,aa)]
        gameOver = b.move(x,y,player)
        if gameOver==-1:
            q[(key,action)]=q[(key,action)]+alpha*(1-q[(key,action)])
            updateModel(model, (key, action), newKey)
            updateModel(model, (newKey, newAction), -b.toKey())
        elif gameOver==1:
            q[(key,action)]=q[(key,action)]+alpha*(0-q[(key,action)])
            updateModel(model, (key, action), -b.toKey())
        elif player==-1 and newAction is not None:
            q[(key,action)]=q[(key,action)]+alpha*(maxq-q[(key,action)])
            updateModel(model, (key, action), newKey)
            action=newAction
            key=newKey

        if player==-1 and newAction is not None:
            for _ in range(nPlanSteps):
                visitedStates = list(model.keys())
                k = visitedStates[np.random.randint(len(visitedStates))]
                dist = model[k]
                d = 0
                for s in dist:
                    d+=dist[s]
                u = np.random.uniform()
                d2=0
                for s in dist:
                    if u < d2+dist[s]/d:
                        state=s
                    else:
                        d2+=dist[s]/d
                if state<0:
                    r = 0
                    hb.fromKey(-state)
                    if hb.check(-1):
                        r = 1
                    state = -state
                else:
                    hb.fromKey(state)
                    pA=hb.possibleActions()
                    maxq=q[(state,pA[0])]
                    for aa in pA[1:]:
                        if q[(state,aa)]>maxq:
                            maxq=q[(state,aa)]
                    r = maxq
                q[k]=q[k]+alpha*(r-q[k])
            
        player=-player

playGame(nGames, q)

100%|██████████| 100000/100000 [00:25<00:00, 3955.12it/s]

0.10859 0.87181 0.0196


In [ ]:
model